# NLP: SMS Spam Classifier

In [101]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


### Loading DataSet

Downloaded from https://archive.ics.uci.edu/dataset/228/sms+spam+collection

In [88]:
messages = pd.read_csv('../helpers/data/SMSSpamCollection', sep='\t', names=['label', 'message'])

In [89]:
messages.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


### Data Cleaning and Preprocessing

In [90]:
ps = PorterStemmer()
corpus = []

for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['message'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [91]:
corpus[:5]

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri wkli comp win fa cup final tkt st may text fa receiv entri question std txt rate c appli',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though']

### Creating the Bag of Words Model

In [92]:
cv = CountVectorizer(max_features=5000)
X = cv.fit_transform(corpus).toarray()
X.shape

(5572, 5000)

In [93]:
y = pd.get_dummies(messages['label']) # Convert labels to one-hot encoding
y = y.iloc[:, 1].values  # Use only the 'spam' column

In [94]:
y[:5]

array([False, False,  True, False, False])

### Training the Model

In [95]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [96]:
classifier = MultinomialNB().fit(X_train, y_train)

In [97]:
y_pred = classifier.predict(X_test)

### Confusion Matrix and Accuracy

In [98]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[943,  12],
       [  8, 152]])

In [99]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 98.21%


It seems that Bag of words model works better than TF-IDF (97.76% accuracy) for this dataset.

In [ ]:
report = classification_report(y_test, y_pred, target_names=['Ham', 'Spam'])
print(report)

              precision    recall  f1-score   support

         Ham       0.99      0.99      0.99       955
        Spam       0.93      0.95      0.94       160

    accuracy                           0.98      1115
   macro avg       0.96      0.97      0.96      1115
weighted avg       0.98      0.98      0.98      1115

